# Performance evaluation

We're going to evaluate portfolio performance. The common way to do this is to estimate a portfolio's return adjusted for risk using a factor model with tradeable risk factors. 

What's a risk factor? These risk factors are portfolios and the idea is that the expected excess return on these risk factors is compensation to investors for bearing the risk inherent in holding those portfolios. For the return variation in these factors to be "risky", it should be something that investors cannot easily diversify. If it was easy to diversify, then investors could put a small bit of the asset in their portfolio and capture the return without affecting portfolio volatility. That would imply being able to increase return without adding risk. Hence the requirement that a factor constitute return variation that is hard to diversify away.

The greater the riskiness of the factor, the greater the factor's expected return (i.e., the risk-return tradeoff). For example, most people feel that stocks are riskier than bonds and indeed stocks have historically had higher returns than bonds.

The risk adjustment involves estimating a portfolio's $\beta$'s on different risk factors. These $\beta$'s constitute the exposure of the portfolio to the risk factor. If the factor return goes up by 1%, then the portfolio's return goes up by $\beta \times 1\%$. 

We can estimate these $\beta$'s by OLS regressions of the portfolio's returns on contemporaneous returns of the risk factors. The slope coefficients on the risk factors are the portfolio's betas on the risk factors. The regression intercept is known as $\alpha$. It represents the average return of the portfolio that is not explained by the portfolio's $\beta$'s on the risk factors. This alpha is the risk-adjusted return. 

Intuitively, $\alpha$ is the average return on a portfolio long the investment you are evaluating and short a portfolio with the same factor risk as that portfolio. If the factors and factor betas accurately measure the portfolio's risk, then the alpha is the portfolio's return that is unrelated to the portfolio's risk. Investors like positive alphas because that implies that the portfolio's return is higher than what investors require for bearing the portfolio's risk.

One thing to keep in mind is that throughout this discussion, we have discussed things from the perspective of arbitrage. That is, like a trader. We have not made any assumptions about utility functions or return distributions. This is the Arbitrage Pricing Theory (APT) of Stephen Ross (1976). He was motivated by the observation that

> "... on theoretical grounds it is difficult to justify either the assumption [in mean-variance anlysis and CAPM] of normality in returns...or of quadratic preferences...and on empirical grounds the conclusions as well as the assumptions of the theory have also come under attack."

The APT way of thinking is less restrictive than economically motivated equilibrium asset pricing models. Which is nice. But it has the cost that it does not tell us as much. With the APT we cannot say precisely what a security's return should be. We can only say that if we go long a portfolio and short the portfolio that replicates its factor exposure, then the alpha shouldn't be *too* big. But if we're thinking like a trader, that's perhaps most of what we care about anyway.




In [ ]:
#r "nuget: FSharp.Stats, 0.4.1"
#r "nuget: FSharp.Data"

#load "../common.fsx"

open System
open FSharp.Data
open Common

open FSharp.Stats

Environment.CurrentDirectory <- __SOURCE_DIRECTORY__


Installed package FSharp.Stats version 0.4.1

Installed package FSharp.Data version 4.1.1

We get the Fama-French 3-Factor asset pricing model data.


In [ ]:
let ff3 = French.getFF3 Frequency.Monthly


Let's get our factor data.


In [ ]:
let myFactorPorts = CsvProvider<"myExcessReturnPortfolios.csv",
                                ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample()


[ML.NET](https://docs.microsoft.com/en-us/dotnet/machine-learning/) is a .NET (C#/F#/VB.NET) machine learning library. There are several [tutorials](https://dotnet.microsoft.com/learn/ml-dotnet) and many F# examples in the sample github repository [here](https://github.com/dotnet/machinelearning-samples/tree/main/samples/fsharp/getting-started).

We will use ML.NET for Ordinary Least Squares (OLS) regression, but you can also do pretty fancy machine learning models with it. So think of it as a gentle introduction giving you some guidance on how to use ML.NET. This will help if you want to experiment with fancy machine learning models after you're done with this course.


In [ ]:
#r "nuget:Microsoft.ML,1.5"
#r "nuget:Microsoft.ML.MKL.Components,1.5"

open Microsoft.ML
open Microsoft.ML.Data


Installed package Microsoft.ML version 1.5.0

Installed package Microsoft.ML.MKL.Components version 1.5.0

Let's start with our long-short portfolio.


In [ ]:
// Get Long Portfolio
let long = myFactorPorts.Rows |> Seq.filter(fun row -> row.PortfolioName = "Mine" && row.Index = Some 3)
// Get Short Portfolio
let short = myFactorPorts.Rows |> Seq.filter(fun row -> row.PortfolioName = "Mine" && row.Index = Some 1)

long |> Seq.take 3

index Item1 Item2 Item3 Item4 0 Mine Value 3 2000-02-01 00:00:00Z -0.023504318580928413 1 Mine Value 3 2000-03-01 00:00:00Z 0.08645743985613241 2 Mine Value 3 2000-04-01 00:00:00Z -0.021817835230565925

In [ ]:
// Get Long Portfolio
let long = myFactorPorts.Rows |> Seq.filter(fun row -> row.PortfolioName = "Mine" && row.Index = Some 3)
// Get Short Portfolio
let short = myFactorPorts.Rows |> Seq.filter(fun row -> row.PortfolioName = "Mine" && row.Index = Some 1)

type Return = { YearMonth : DateTime; Return : float }
let longShort =
    // this is joining long to short by YearMonth:DateTime and we get the whole row for it
    let shortMap = short |> Seq.map(fun row -> row.YearMonth, row) |> Map
    long
    |> Seq.map(fun longObs -> 
        match Map.tryFind longObs.YearMonth shortMap with
        | None -> failwith "probably your date variables are not aligned"
        | Some shortObs -> { YearMonth = longObs.YearMonth; Return = longObs.Ret - shortObs.Ret })
    |> Seq.toArray    
        
longShort |> Seq.take 5

index,YearMonth,Return
0,2000-02-01 00:00:00Z,-0.09159469165540102
1,2000-03-01 00:00:00Z,0.008860871744062651
2,2000-04-01 00:00:00Z,0.07751895876386396
3,2000-05-01 00:00:00Z,0.04945087705506296
4,2000-06-01 00:00:00Z,-0.059241392068224354


For regression, it is helpful to have the portfolio
return data merged into our factor model data.


In [ ]:
type RegData =
    // The ML.NET OLS trainer requires 32bit "single" floats
    { Date : DateTime
      Portfolio : single
      MktRf : single 
      Hml : single 
      Smb : single }

// ff3 indexed by month
// We're not doing date arithmetic, so I'll just
// use DateTime on the 1st of the month to represent a month
let ff3ByMonth = 
    ff3
    |> Array.map(fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

let longShortRegData =
    longShort 
    |> Array.map(fun port ->
        let monthToFind = DateTime(port.YearMonth.Year,port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = single port.Return // single converts to 32bit
              MktRf = single ff3.MktRf 
              Hml = single ff3.Hml 
              Smb = single ff3.Smb })


In [ ]:
ff3ByMonth |> Seq.take 2

index Key Value 0 1926-07-01 00:00:00Z Date MktRf Smb Hml Rf Frequency 1926-07-01 00:00:00Z 0.0296 -0.023 -0.0287 0.0022 Monthly 1 1926-08-01 00:00:00Z Date MktRf Smb Hml Rf Frequency 1926-08-01 00:00:00Z 0.0264 -0.013999999999999999 0.04190000000000001 0.0025 Monthly

In [ ]:
longShortRegData |> Seq.take 3

index,Date,Portfolio,MktRf,Hml,Smb
0,2000-02-01 00:00:00Z,-0.09159469,0.0245,-0.0894,0.2119
1,2000-03-01 00:00:00Z,0.008860872,0.052,0.0766,-0.1682
2,2000-04-01 00:00:00Z,0.07751896,-0.064,0.0741,-0.0693


We need to define a ML.Net ["context"](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.mlcontext?view=ml-dotnet)

> Once instantiated by the user, it provides a way to create components for data preparation, feature enginering, training, prediction, model evaluation.



In [ ]:
let ctx = new MLContext()


Now we can use the context to transform the data into ML.NET's format.

In the below code, a .NET Enumerable collection is equivalent to an F# sequence.
This line says load an F# collection were the elements of the collection are `RegData` records.
The part between `<>` is how we define the type of the data on the collection.


In [ ]:
let longShortMlData = ctx.Data.LoadFromEnumerable<RegData>(longShortRegData)


Now we are going to define our machine learning trainer. OLS!

The OLS trainer is documented [here](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.mklcomponentscatalog.ols?view=ml-dotnet#Microsoft_ML_MklComponentsCatalog_Ols_Microsoft_ML_RegressionCatalog_RegressionTrainers_Microsoft_ML_Trainers_OlsTrainer_Options_) with an example in C#. Though C# is not the easiest language to follow.

But [these](https://github.com/dotnet/machinelearning-samples/blob/main/samples/fsharp/getting-started/Regression_BikeSharingDemand/BikeSharingDemand/BikeSharingDemandConsoleApp/Program.fs) F# regression examples with fancier ML models is easier. So we do the OLS trainer like they do those trainers.



In [ ]:
let trainer = ctx.Regression.Trainers.Ols()


Now we define the models that we want to estimate.
Think of this like an ML pipeline that chains data prep and model estimation.

- `Label` is the variable that we are trying to predict or explain with our model.
- `Features` are the variables that we are using to predict the label column.


In [ ]:
let capmModel = 
    EstimatorChain()
        .Append(ctx.Transforms.CopyColumns("Label","Portfolio"))
        .Append(ctx.Transforms.Concatenate("Features",[|"MktRf"|])) 
        .Append(trainer)   

let ff3Model =
    EstimatorChain()
        .Append(ctx.Transforms.CopyColumns("Label","Portfolio"))
        .Append(ctx.Transforms.Concatenate("Features",[|"MktRf";"Hml";"Smb"|]))
        .Append(trainer)   


Now we can estimate our models.


In [ ]:
let capmEstimate = longShortMlData |> capmModel.Fit
let ff3Estimate = longShortMlData |> ff3Model.Fit


The results can be found in [OLSModelParameters Class](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.trainers.olsmodelparameters?view=ml-dotnet).
CAPM results.

In [ ]:
capmEstimate.LastTransformer.Model

HasStatistics,StandardErrors,TValues,PValues,Weights,Bias,RSquared,RSquaredAdjusted
True,"[ 0.0014720679927722707, 0.03204461466705298 ]","[ 2.736204600831987, -5.677019539128863 ]","[ 0.006662753410637379, 3.804176174071472E-08 ]",[ -0.1819179 ],0.0040278793,0.1145991516612822,0.11104332496112679


Fama-French 3-Factor model results


In [ ]:
ff3Estimate.LastTransformer.Model


HasStatistics,StandardErrors,TValues,PValues,Weights,Bias,RSquared,RSquaredAdjusted
True,"[ 0.0016134248861530685, 0.03678921856345468, 0.05010132774887472, 0.05306389205511768 ]","[ 2.761917431665564, -12.710616665687168, 7.707783082678414, -1.2397767932674972 ]","[ 0.0061780852265655994, 8.165679352540742E-29, 3.119433964610119E-13, 0.21623453497886658 ]","[ -0.46761367, 0.38617018, -0.06578738 ]",0.004456146,0.5089703597941682,0.5030064370386318


For your portfolio, multiply the Alpha (BIAS) by 12 to get the annual alpha, but DO NOT multiply the t-stat

You will probably see that the CAPM $R^2$ is lower than the
Fama-French $R^2$. This means that you can explain more of the
portfolio's returns with the Fama-French model. Or in trader terms,
you can hedge the portfolio better with the multi-factor model.
We also want predicted values so that we can get regression residuals for calculating
the information ratio. ML.NET calls the predicted value the [score](https://docs.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/machine-learning-model-predictions-ml-net).

The ML.NET OLS example shows getting predicted values using [C#](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.mklcomponentscatalog.ols?view=ml-dotnet#Microsoft_ML_MklComponentsCatalog_Ols_Microsoft_ML_RegressionCatalog_RegressionTrainers_Microsoft_ML_Trainers_OlsTrainer_Options_) with the `context.Data.CreateEnumarable`. Searching the ML.NET samples github repo for `CreateEnumerable` shows [F#](https://github.com/dotnet/machinelearning-samples/search?l=F%23&q=createenumerable) examples.


In [ ]:
[<CLIMutable>]
type Prediction = { Label : single; Score : single}

let makePredictions (estimate:TransformerChain<_>) data =
    ctx.Data.CreateEnumerable<Prediction>(estimate.Transform(data),reuseRowObject=false)
    |> Seq.toArray

makePredictions capmEstimate longShortMlData
|> Array.take 5

index,Label,Score
0,-0.09159469,-0.0004291092
1,0.008860872,-0.005431852
2,0.07751896,0.015670625
3,0.049450878,0.01206865
4,-0.05924139,-0.0044131116


In [ ]:
[<CLIMutable>]
type Prediction = { Label : single; Score : single}

let makePredictions (estimate:TransformerChain<_>) data =
    ctx.Data.CreateEnumerable<Prediction>(estimate.Transform(data),reuseRowObject=false)
    |> Seq.toArray

let residuals (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)

let capmPredictions = makePredictions capmEstimate longShortMlData
let ff3Predictions = makePredictions ff3Estimate longShortMlData

capmPredictions |> Array.take 3


index,Label,Score
0,-0.09159469,-0.0004291092
1,0.008860872,-0.005431852
2,0.07751896,0.015670625


In [ ]:
capmPredictions |> residuals |> Array.take 3


index,value
0,-0.09116558
1,0.014292724
2,0.061848335


In [ ]:
let capmResiduals = residuals capmPredictions
let ff3Residuals = residuals ff3Predictions


In general I would write a function to do this. Function makes it a bit
simpler to follow. It's hard for me to read the next few lines and understand
what everything is. Too much going on.


In [ ]:
// Single is the type of the float
let capmAlpha = (single 12.0) * capmEstimate.LastTransformer.Model.Bias 
capmAlpha

0.048334554

In [ ]:
// Get annualized Stuff, Alpha, StdResiduals and Information Ratio
let capmAlpha = (single 12.0) * capmEstimate.LastTransformer.Model.Bias 
let capmStDevResiduals = sqrt(single 12) * (Seq.stDev capmResiduals)
let capmInformationRatio = capmAlpha / capmStDevResiduals


In [ ]:
let ff3Alpha = (single 12.0) * ff3Estimate.LastTransformer.Model.Bias 
let ff3StDevResiduals = sqrt(single 12) * (Seq.stDev ff3Residuals)
let ff3InformationRatio = ff3Alpha / ff3StDevResiduals


In [ ]:
// Function version

let informationRatio monthlyAlpha (monthlyResiduals: single array) =
    let annualAlpha = single 12.0 * monthlyAlpha
    let annualStDev = sqrt(single 12.0) * (Seq.stDev monthlyResiduals)
    annualAlpha / annualStDev 

informationRatio capmEstimate.LastTransformer.Model.Bias capmResiduals


0.60413414

In [ ]:
informationRatio ff3Estimate.LastTransformer.Model.Bias ff3Residuals


0.65950793